In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from datetime import datetime

# Startup

- Anthony
  - Linear Regression
  - Analytic goal: Assess whether viewership is affected by public sentiment
- William
  - Random Forest
  - Analytic goal: Assess whether viewership is affected by public sentiment
- Lawrence
  - Gradient Boosted Trees 
  - Analytic goal: Assess whether viewership is affected by public sentiment

In [0]:
def disp_normalize_mse(mse, divisor):
    print(f"Normalized MSE: {mse/divisor:.3f}")

In [0]:
import time
start = time.time()

In [0]:
ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext

In [0]:
ss = SparkSession.builder.appName("rogan_sentiment_analysis")\
                            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
                            .config("spark.network.timeout", "36000000s")\
                            .config("spark.executor.heartbeatInterval", "3600s")\
                            .getOrCreate()
sc = spark.sparkContext

In [0]:
aws_access_key = ACCESS_KEY
aws_secret_key = SECRET_KEY
spark._jsc.hadoopConfiguration().set(
    "spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1"
)
spark._jsc.hadoopConfiguration().set(
    "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
)
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)

In [0]:
database = DATABASE
collection = COLLECTION
user_name = USER_NAME
password = PASSWORD
address = ADDRESS
connection_string = (
    f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
)

In [0]:
rogan_schema = StructType(
    [
        StructField("week", DateType(), False),
        StructField("views_gained", IntegerType(), False),
        StructField("date", DateType(), False),
        StructField("text", StringType(), True),
        StructField("likes", IntegerType(), True),
        StructField("text_length", IntegerType(), True),
        StructField("hf_emot_label", StringType(), True),
        StructField("vader_sentiment_score", FloatType(), True)
    ]
)

In [0]:
database = DATABASE
collection = COLLECTION
user_name = USER_NAME
password = PASSWORD
address = ADDRESS
connection_string = (
    f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
)

In [0]:
df = spark.read.format("mongo").option("uri", connection_string).load()
df.cache()
df.show()

+--------------------+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
 _id| date|hf_emot_label|likes| text|text_length|vader_sentiment_score|views_gained| week|
+--------------------+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 1|I enjoy Joe Rogan...| 88| 0.784500002861023| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|@JohnMcCarthyMMA ...| 76| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 1|@cenkuygur Chunk ...| 162| -0.6820999979972839| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| sadness| 0|@CaveDude111 He's...| 181| 0.16220000386238098| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| sadness| 0|@cenkuygur @joero...| 45| -0.5719000101089478| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 1|@cenkuygur @joero...| 103| 0.5156999826431274| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| sadness| 1|Finally a man wit...| 178| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| surprise| 1|@cenkuygur @joero...| 144| 0.883400022983551| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|@alex_ozarud @ela...| 101| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 4|@BridgetPhetasy B...| 159| -0.3253999948501587| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 0|@Tlove4ucla @Real...| 41| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 0|@JohnMcCarthyMMA ...| 91| -0.2410999983549118| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 1|Sign this shit up...| 162| 0.27140000462532043| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|@NicoleArbour @st...| 65| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|Please let this h...| 115| 0.3181999921798706| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|@JohnMcCarthyMMA ...| 47| 0.29600000381469727| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 1|@cenkuygur @joero...| 82| 0.0| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 8|@cenkuygur @joero...| 69| -0.45879998803138733| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| anger| 0|@RawStory @joerog...| 31| -0.3612000048160553| 8910349|2021-09-27 07:00:00|
{622c4e2d842a9536...|2021-09-30 07:00:00| joy| 0|@NoronhaBrennan @...| 56| 0.6596999764442444| 8910349|2021-09-27 07:00:00|
+--------------------+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
only showing top 20 rows

In [0]:
df = df.drop("_id")

In [0]:
df.printSchema()

root
-- date: timestamp (nullable = true)
-- hf_emot_label: string (nullable = true)
-- likes: integer (nullable = true)
-- text: string (nullable = true)
-- text_length: integer (nullable = true)
-- vader_sentiment_score: double (nullable = true)
-- views_gained: integer (nullable = true)
-- week: timestamp (nullable = true)

In [0]:
view_mean = df.agg({"views_gained":"mean"})

In [0]:
view_mean = view_mean.collect()

In [0]:
view_mean = list(view_mean[0].asDict().values())[0]

In [0]:
df.show()

+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
 date|hf_emot_label|likes| text|text_length|vader_sentiment_score|views_gained| week|
+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
2021-09-30 07:00:00| joy| 1|I enjoy Joe Rogan...| 88| 0.784500002861023| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|@JohnMcCarthyMMA ...| 76| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 1|@cenkuygur Chunk ...| 162| -0.6820999979972839| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| sadness| 0|@CaveDude111 He's...| 181| 0.16220000386238098| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| sadness| 0|@cenkuygur @joero...| 45| -0.5719000101089478| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 1|@cenkuygur @joero...| 103| 0.5156999826431274| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| sadness| 1|Finally a man wit...| 178| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| surprise| 1|@cenkuygur @joero...| 144| 0.883400022983551| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|@alex_ozarud @ela...| 101| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 4|@BridgetPhetasy B...| 159| -0.3253999948501587| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| joy| 0|@Tlove4ucla @Real...| 41| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| joy| 0|@JohnMcCarthyMMA ...| 91| -0.2410999983549118| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| joy| 1|Sign this shit up...| 162| 0.27140000462532043| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|@NicoleArbour @st...| 65| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|Please let this h...| 115| 0.3181999921798706| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|@JohnMcCarthyMMA ...| 47| 0.29600000381469727| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| joy| 1|@cenkuygur @joero...| 82| 0.0| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 8|@cenkuygur @joero...| 69| -0.45879998803138733| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| anger| 0|@RawStory @joerog...| 31| -0.3612000048160553| 8910349|2021-09-27 07:00:00|
2021-09-30 07:00:00| joy| 0|@NoronhaBrennan @...| 56| 0.6596999764442444| 8910349|2021-09-27 07:00:00|
+-------------------+-------------+-----+--------------------+-----------+---------------------+------------+-------------------+
only showing top 20 rows

In [0]:
df = df.drop('week').drop('date').drop('text')

In [0]:
df.show()

+-------------+-----+-----------+---------------------+------------+
hf_emot_label|likes|text_length|vader_sentiment_score|views_gained|
+-------------+-----+-----------+---------------------+------------+
 joy| 1| 88| 0.784500002861023| 8910349|
 anger| 0| 76| 0.0| 8910349|
 anger| 1| 162| -0.6820999979972839| 8910349|
 sadness| 0| 181| 0.16220000386238098| 8910349|
 sadness| 0| 45| -0.5719000101089478| 8910349|
 anger| 1| 103| 0.5156999826431274| 8910349|
 sadness| 1| 178| 0.0| 8910349|
 surprise| 1| 144| 0.883400022983551| 8910349|
 anger| 0| 101| 0.0| 8910349|
 anger| 4| 159| -0.3253999948501587| 8910349|
 joy| 0| 41| 0.0| 8910349|
 joy| 0| 91| -0.2410999983549118| 8910349|
 joy| 1| 162| 0.27140000462532043| 8910349|
 anger| 0| 65| 0.0| 8910349|
 anger| 0| 115| 0.3181999921798706| 8910349|
 anger| 0| 47| 0.29600000381469727| 8910349|
 joy| 1| 82| 0.0| 8910349|
 anger| 8| 69| -0.45879998803138733| 8910349|
 anger| 0| 31| -0.3612000048160553| 8910349|
 joy| 0| 56| 0.6596999764442444| 8910349|
+-------------+-----+-----------+---------------------+------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
df_split = df.randomSplit([0.8, 0.2], 1)
df_train = df_split[0].cache()
df_test = df_split[1].cache()

In [0]:
df_train.show()

+-------------+-----+-----------+---------------------+------------+
hf_emot_label|likes|text_length|vader_sentiment_score|views_gained|
+-------------+-----+-----------+---------------------+------------+
 anger| -1| 0| 0.0| 8096839|
 anger| -1| 0| 0.0| 9179147|
 anger| -1| 0| 0.0| 14023547|
 anger| -1| 0| 0.0| 14045687|
 anger| -1| 0| 0.0| 14124456|
 anger| -1| 1| 0.0| 10782156|
 anger| -1| 1| 0.0| 14124456|
 anger| -1| 3| 0.0| 20157423|
 anger| -1| 4| 0.0| 11203646|
 anger| -1| 4| 0.0| 12242089|
 anger| -1| 4| 0.0| 14126353|
 anger| -1| 5| 0.0| 6401358|
 anger| -1| 5| 0.6359999775886536| 17233953|
 anger| -1| 6| 0.0| 6401358|
 anger| -1| 6| 0.0| 9179147|
 anger| -1| 6| 0.0| 9179147|
 anger| -1| 6| 0.0| 12757312|
 anger| -1| 6| 0.0| 14770807|
 anger| -1| 6| 0.5573999881744385| 19104055|
 anger| -1| 7| -0.4018999934196472| 9179147|
+-------------+-----+-----------+---------------------+------------+
only showing top 20 rows

In [0]:
cat_cols = ['hf_emot_label']

for col in cat_cols:
    si = StringIndexer(inputCol=col, outputCol=col+"-onehot").setHandleInvalid("keep")
    sm = si.fit(df_train)
    df_train = sm.transform(df_train).drop(col)
    df_train = df_train.withColumnRenamed(col+"-onehot", col)
    df_test = sm.transform(df_test).drop(col)
    df_test = df_test.withColumnRenamed(col+"-onehot", col)

In [0]:
sm.labels

Out[63]: ['joy', 'anger', 'sadness', 'fear', 'surprise', 'love']

In [0]:
df_train.show()

+-----+-----------+---------------------+------------+-------------+
likes|text_length|vader_sentiment_score|views_gained|hf_emot_label|
+-----+-----------+---------------------+------------+-------------+
 -1| 0| 0.0| 8096839| 1.0|
 -1| 0| 0.0| 9179147| 1.0|
 -1| 0| 0.0| 14023547| 1.0|
 -1| 0| 0.0| 14045687| 1.0|
 -1| 0| 0.0| 14124456| 1.0|
 -1| 1| 0.0| 10782156| 1.0|
 -1| 1| 0.0| 14124456| 1.0|
 -1| 3| 0.0| 20157423| 1.0|
 -1| 4| 0.0| 11203646| 1.0|
 -1| 4| 0.0| 12242089| 1.0|
 -1| 4| 0.0| 14126353| 1.0|
 -1| 5| 0.0| 6401358| 1.0|
 -1| 5| 0.6359999775886536| 17233953| 1.0|
 -1| 6| 0.0| 6401358| 1.0|
 -1| 6| 0.0| 9179147| 1.0|
 -1| 6| 0.0| 9179147| 1.0|
 -1| 6| 0.0| 12757312| 1.0|
 -1| 6| 0.0| 14770807| 1.0|
 -1| 6| 0.5573999881744385| 19104055| 1.0|
 -1| 7| -0.4018999934196472| 9179147| 1.0|
+-----+-----------+---------------------+------------+-------------+
only showing top 20 rows

In [0]:
for col in cat_cols:
    oi = OneHotEncoder(inputCol=col, outputCol=col+"-onehot")
    om = oi.fit(df_train)
    df_train = om.transform(df_train).drop(col)
    df_train = df_train.withColumnRenamed(col+"-onehot", col)
    df_test = om.transform(df_test).drop(col)
    df_test = df_test.withColumnRenamed(col+"-onehot", col)

In [0]:
om.categorySizes

Out[62]: [7]

In [0]:
df_train.show()

+-----+-----------+---------------------+------------+-------------+
likes|text_length|vader_sentiment_score|views_gained|hf_emot_label|
+-----+-----------+---------------------+------------+-------------+
 -1| 0| 0.0| 8096839|(6,[1],[1.0])|
 -1| 0| 0.0| 9179147|(6,[1],[1.0])|
 -1| 0| 0.0| 14023547|(6,[1],[1.0])|
 -1| 0| 0.0| 14045687|(6,[1],[1.0])|
 -1| 0| 0.0| 14124456|(6,[1],[1.0])|
 -1| 1| 0.0| 10782156|(6,[1],[1.0])|
 -1| 1| 0.0| 14124456|(6,[1],[1.0])|
 -1| 3| 0.0| 20157423|(6,[1],[1.0])|
 -1| 4| 0.0| 11203646|(6,[1],[1.0])|
 -1| 4| 0.0| 12242089|(6,[1],[1.0])|
 -1| 4| 0.0| 14126353|(6,[1],[1.0])|
 -1| 5| 0.0| 6401358|(6,[1],[1.0])|
 -1| 5| 0.6359999775886536| 17233953|(6,[1],[1.0])|
 -1| 6| 0.0| 6401358|(6,[1],[1.0])|
 -1| 6| 0.0| 9179147|(6,[1],[1.0])|
 -1| 6| 0.0| 9179147|(6,[1],[1.0])|
 -1| 6| 0.0| 12757312|(6,[1],[1.0])|
 -1| 6| 0.0| 14770807|(6,[1],[1.0])|
 -1| 6| 0.5573999881744385| 19104055|(6,[1],[1.0])|
 -1| 7| -0.4018999934196472| 9179147|(6,[1],[1.0])|
+-----+-----------+---------------------+------------+-------------+
only showing top 20 rows

In [0]:
input_cols = df_train.columns
input_cols.remove('views_gained')
va = VectorAssembler(outputCol="features", inputCols=input_cols)

In [0]:
input_cols

Out[53]: ['likes', 'text_length', 'vader_sentiment_score', 'hf_emot_label']

In [0]:
df_train = va.transform(df_train).select(
    "features", "views_gained").withColumnRenamed("views_gained", "label")
df_test = va.transform(df_test).select(
    "features", "views_gained").withColumnRenamed("views_gained", "label")

In [0]:
df_train.show(truncate=False)

+------------------------------------------------+--------+
features |label |
+------------------------------------------------+--------+
(9,[0,4],[-1.0,1.0]) |8096839 |
(9,[0,4],[-1.0,1.0]) |9179147 |
(9,[0,4],[-1.0,1.0]) |14023547|
(9,[0,4],[-1.0,1.0]) |14045687|
(9,[0,4],[-1.0,1.0]) |14124456|
(9,[0,1,4],[-1.0,1.0,1.0]) |10782156|
(9,[0,1,4],[-1.0,1.0,1.0]) |14124456|
(9,[0,1,4],[-1.0,3.0,1.0]) |20157423|
(9,[0,1,4],[-1.0,4.0,1.0]) |11203646|
(9,[0,1,4],[-1.0,4.0,1.0]) |12242089|
(9,[0,1,4],[-1.0,4.0,1.0]) |14126353|
(9,[0,1,4],[-1.0,5.0,1.0]) |6401358 |
(9,[0,1,2,4],[-1.0,5.0,0.6359999775886536,1.0]) |17233953|
(9,[0,1,4],[-1.0,6.0,1.0]) |6401358 |
(9,[0,1,4],[-1.0,6.0,1.0]) |9179147 |
(9,[0,1,4],[-1.0,6.0,1.0]) |9179147 |
(9,[0,1,4],[-1.0,6.0,1.0]) |12757312|
(9,[0,1,4],[-1.0,6.0,1.0]) |14770807|
(9,[0,1,2,4],[-1.0,6.0,0.5573999881744385,1.0]) |19104055|
(9,[0,1,2,4],[-1.0,7.0,-0.4018999934196472,1.0])|9179147 |
+------------------------------------------------+--------+
only showing top 20 rows

In [0]:
rf = RandomForestRegressor(maxDepth=3, numTrees=20, minInstancesPerNode=2)
rfmodel = rf.fit(df_train)
rfpredicts = rfmodel.transform(df_test)
metric_name = "mse"
metrics = RegressionEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName(metric_name) 

rf_mse = metrics.evaluate(rfpredicts)
disp_normalize_mse(rf_mse, view_mean)

Normalized MSE: 846704.339

In [0]:
rfpredicts.show()

+--------------------+--------+--------------------+
 features| label| prediction|
+--------------------+--------+--------------------+
(9,[0,4],[-1.0,1.0])|14124456|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|12242089|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14770807|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|17939800|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 6401358|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 9656136|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14126353|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|19104055|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 6401358|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 6401358|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 7612963|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 8096839|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 9179147|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...| 9413514|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|12029950|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|12029950|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14045687|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14045687|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14126353|1.2415377087236244E7|
(9,[0,1,4],[-1.0,...|14770807|1.2415377087236244E7|
+--------------------+--------+--------------------+
only showing top 20 rows

In [0]:
rfmodel.featureImportances

Out[48]: SparseVector(9, {0: 0.5052, 1: 0.1427, 2: 0.0687, 3: 0.1122, 4: 0.0745, 5: 0.0832, 6: 0.0078, 7: 0.0021, 8: 0.0035})

# Gradient boosted MSE

In [0]:
gb = GBTRegressor()
gb = GBTRegressor(maxDepth=2, maxIter=10, stepSize=0.01)
gbmodel = gb.fit(df_train)
gbpredicts = gbmodel.transform(df_test)
metric_name = "mse"
metrics = RegressionEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName(metric_name) 

gb_mse = metrics.evaluate(gbpredicts)
disp_normalize_mse(gb_mse, view_mean)

Normalized MSE: 846695.766

In [0]:
gbmodel.featureImportances 

Out[44]: SparseVector(9, {0: 0.4017, 1: 0.2927, 3: 0.0288, 5: 0.2768})

# Linear Regression

In [0]:
df_train.show(3)

+--------------------+--------+
 features| label|
+--------------------+--------+
(9,[0,4],[-1.0,1.0])| 8096839|
(9,[0,4],[-1.0,1.0])| 9179147|
(9,[0,4],[-1.0,1.0])|14023547|
+--------------------+--------+
only showing top 3 rows

In [0]:
lr = LinearRegression()
lrmodel = lr.fit(df_train)

trainingSummary = lrmodel.summary

In [0]:
lrmodel.coefficients

Out[42]: DenseVector([28.5988, -224.2093, 19472.3817, 64417.1469, -28594.4941, -265221.8078, -154440.4623, 16568.3813, -294766.0986])

In [0]:
lr_mse = trainingSummary.meanSquaredError

In [0]:
disp_normalize_mse(lr_mse, view_mean)

Normalized MSE: 846779.410

In [0]:
end = time.time()
print(f"{end - start:.3f} seconds to train and evaluate models")

267.802 seconds to train and evaluate models

In [0]:
sc.stop()
ss.stop()